In [7]:
# !pip3 install tensorflow-datasets

In [8]:
import tensorflow as tf
import tensorflow_datasets as tfds
import pyspark

In [9]:
spark = pyspark.sql.SparkSession.builder.appName("MyApp") \
            .getOrCreate()

22/02/22 06:41:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [10]:
mnist, info = tfds.load('mnist', with_info=True)
print(info.as_json)

Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.
{
  "citation": "@article{lecun2010mnist,\n  title={MNIST handwritten digit database},\n  author={LeCun, Yann and Cortes, Corinna and Burges, CJ},\n  journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},\n  volume={2},\n  year={2010}\n}",
  "description": "The MNIST database of handwritten digits.",
  "downloadSize": "11594722",
  "fileFormat": "tfrecord",
  "location": {
    "urls": [
      "http://yann.lecun.com/exdb/mnist/"
    ]
  },
  "moduleName": "tensorflow_datasets.image_classification.mnist",
  "name": "mnist",
  "schema": {
    "feature": [
      {
        "name": "image",
        "shape": {
          "dim": [
            {
              "size": "28"
            },
            {
              "size": "28"
            },
            {
              "size": "1"
            }
          ]
        },
        "type": "INT"
      },
      {
    

2022-02-22 06:41:19.570620: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-22 06:41:19.570733: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-22 06:41:19.570786: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (0905a014a175): /proc/driver/nvidia/version does not exist
2022-02-22 06:41:19.573120: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
output = 'hdfs://hdfs:9000/user/mnist'
num_partitions = 10

In [12]:
mnist_train = tfds.as_numpy(mnist['train'])
mnist_test = tfds.as_numpy(mnist['test'])

train_rdd = spark.sparkContext.parallelize(mnist_train, num_partitions).cache()
test_rdd = spark.sparkContext.parallelize(mnist_test, num_partitions).cache()

2022-02-22 06:41:19.719347: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-02-22 06:41:19.720597: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2199995000 Hz


In [16]:
# !$HADOOP_HOME/bin/hdfs dfs -rm -r hdfs://hdfs:9000/user/mnist/csv

In [17]:
def to_csv(example):
    return str(example['label']) + ',' + ','.join([str(i) for i in example['image'].reshape(784)])

train_rdd.map(to_csv).saveAsTextFile(output + "/csv/train")
test_rdd.map(to_csv).saveAsTextFile(output + "/csv/test")

22/02/22 06:42:40 WARN TaskSetManager: Stage 0 contains a task of very large size (4205 KiB). The maximum recommended task size is 1000 KiB.


In [18]:
def to_tfr(example):
    ex = tf.train.Example(
      features=tf.train.Features(
        feature={
          'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[example['label'].astype("int64")])),
          'image': tf.train.Feature(int64_list=tf.train.Int64List(value=example['image'].reshape(784).astype("int64")))
        }
      )
    )
    return (bytearray(ex.SerializeToString()), None)

train_rdd.map(to_tfr).saveAsNewAPIHadoopFile(output + "/tfr/train",
                                           "org.tensorflow.hadoop.io.TFRecordFileOutputFormat",
                                           keyClass="org.apache.hadoop.io.BytesWritable",
                                           valueClass="org.apache.hadoop.io.NullWritable")
test_rdd.map(to_tfr).saveAsNewAPIHadoopFile(output + "/tfr/test",
                                           "org.tensorflow.hadoop.io.TFRecordFileOutputFormat",
                                           keyClass="org.apache.hadoop.io.BytesWritable",
                                           valueClass="org.apache.hadoop.io.NullWritable")

22/02/22 06:44:16 WARN TaskSetManager: Stage 2 contains a task of very large size (4205 KiB). The maximum recommended task size is 1000 KiB.
2022-02-22 06:44:16.827356: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-22 06:44:16.827848: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
22/02/22 06:44:18 WARN TaskSetManager: Stage 3 contains a task of very large size (4205 KiB). The maximum recommended task size is 1000 KiB.
2022-02-22 06:44:19.134025: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-22 06:44:19.134104: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart